# Unit Model Configuration

When undertaking a modeling exercise for a particular process, there can be any number of different approaches, assumptions, or relevant phenomena to include (or not include) in the model that is used. In WaterTAP, these discrete decisions are made at the point of model instantiation via passing of _configuration arguments_.

Passing different configuration arguments will typically result in the creation (or exclusion) of specific variables and constraints.

Some models have many configuration arguments and some have none. All WaterTAP unit process models require passing the required property model via the `property_package` configuration argument. This tutorial will go through configuration arguments for the WaterTAP pump and RO model.

<center><img src="graphics/config_arguments.png" width="700" /></center>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Imports from Pyomo
from pyomo.environ import (
    ConcreteModel,
    value,
    assert_optimal_termination,
    units as pyunits,
)

# Imports from IDAES
from idaes.core import FlowsheetBlock
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor

# Imports from WaterTAP
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock

from watertap.unit_models import Pump, ReverseOsmosis0D
from watertap.core.solvers import get_solver

solver = get_solver()

# Pump Model & Configuration

### We will use the following parameters in our pump models

In [ ]:
# Pump parameters
eta = 0.8
bep_flow = flow = 1e-3
salinity = 35  # g/L
temperature = 298  # K
inlet_pressure = 40 * pyunits.bar
deltaP = 10 * pyunits.bar

flows = np.linspace(6e-4, bep_flow, 25)  # m3/s

### Create model, flowsheet, and add property model

Every WaterTAP model will begin with these three steps. 

The first flowsheet we are going to build in this tutorial is a pump without variable efficiency.

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

### Import pump and add to flowsheet


In [ ]:
from watertap.unit_models import Pump

# Add pump model to flowsheet
m.fs.pump = Pump(property_package=m.fs.properties)

# View configuration options
m.fs.pump.config.display()

### Fix inlet conditions


In [ ]:
# Fix inlet conditions
m.fs.pump.control_volume.properties_in[0].flow_vol_phase["Liq"].fix(flow)
m.fs.pump.control_volume.properties_in[0].conc_mass_phase_comp["Liq", "TDS"].fix(
    salinity
)
m.fs.pump.control_volume.properties_in[0].pressure.fix(inlet_pressure)
m.fs.pump.control_volume.properties_in[0].temperature.fix(temperature)

### Fix pump performance parameters


In [ ]:
# Fix pump parameters
m.fs.pump.efficiency_pump.fix(eta)
m.fs.pump.deltaP.fix(deltaP)

### Solve the model


In [ ]:
# Check degrees of freedom
assert degrees_of_freedom(m.fs.pump) == 0
# Solve model
results = solver.solve(m)
assert_optimal_termination(results)
m.fs.pump.work_mechanical.display()

## Build pump model with variable efficiency

In [ ]:
##################################################################
# Build Pump with variable_efficiency

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

### Configuring the variable efficiency pump

In [ ]:
# Add pump model to flowsheet
# Set variable_efficiency to "flow"
m.fs.pump = Pump(property_package=m.fs.properties, variable_efficiency="flow")

### Fix inlet conditions and pump performance parameters and solve

In [ ]:
# Fix inlet conditions
m.fs.pump.control_volume.properties_in[0].flow_vol_phase["Liq"].fix(flow)
m.fs.pump.control_volume.properties_in[0].conc_mass_phase_comp["Liq", "TDS"].fix(
    salinity
)
m.fs.pump.control_volume.properties_in[0].pressure.fix(inlet_pressure)
m.fs.pump.control_volume.properties_in[0].temperature.fix(temperature)

# Fix pump parameters
m.fs.pump.deltaP.fix(deltaP)
m.fs.pump.bep_eta.fix(eta)
m.fs.pump.bep_flow.fix(bep_flow)

### Solve to determine pump efficiency and power required

In [ ]:
# Solve model
assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

print(f"Pump efficiency: {value(m.fs.pump.efficiency_pump[0]):.4f}")
print(f"Pump power: {value(m.fs.pump.work_mechanical[0]):.4f}")

# RO Model & Configuration


### *Simple*
```python
concentration_polarization_type="none"
mass_transfer_coefficient="none"
pressure_change_type="fixed_per_stage"
transport_model="SD"
module_type="spiral_wound"
```
### *Complex*
```python
concentration_polarization_type="calculated"
mass_transfer_coefficient="calculated"
pressure_change_type="calculated"
transport_model="SD"
module_type="spiral_wound"
```

### We will use the following parameters in our RO models

In [ ]:
inlet_pressure = 50 * pyunits.bar
temperature = 298  # K
A_comp = 4.2e-12
B_comp = 3e-8
membrane_area = 50  # m2
atmospheric = 101325  # Pa
deltaP = -3 * pyunits.bar
channel_height = 1 * pyunits.mm
spacer_porosity = 0.75

### RO Simple Model Build

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="none",
    mass_transfer_coefficient="none",
    has_pressure_change=False,
    transport_model="SD",
    module_type="spiral_wound",
)

### Fix state variables

In [ ]:
# m.fs.RO.feed_side.properties_in[0].flow_mass_phase_comp["Liq", "TDS"].fix(0.035)
# m.fs.RO.feed_side.properties_in[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.965)
# m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
# m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.temperature[0].fix(temperature)
m.fs.RO.inlet.pressure[0].fix(inlet_pressure)

### Fix process variables


In [ ]:
m.fs.RO.permeate.pressure[0].fix(atmospheric)
m.fs.RO.area.fix(membrane_area)
m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)

print(f"Degrees of freedom: {degrees_of_freedom(m)}")

### Scale the model

In [ ]:
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)
calculate_scaling_factors(m)

### Initialize & solve


In [ ]:
m.fs.RO.initialize()

assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

### Complex RO model build

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="calculated",
    mass_transfer_coefficient="calculated",
    has_pressure_change=True,
    pressure_change_type="calculated",
    transport_model="SD",
    module_type="spiral_wound",
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)

m.fs.RO.permeate_side[0, 0].pressure.fix(atmospheric)
m.fs.RO.area.fix(membrane_area)
m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)
# Additional degrees of freedom from complex build
m.fs.RO.deltaP.fix(deltaP)
m.fs.RO.feed_side.channel_height.fix(channel_height)
m.fs.RO.feed_side.spacer_porosity.fix(spacer_porosity)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)
calculate_scaling_factors(m)

m.fs.RO.initialize()

assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

# Try It Yourself

Create a complex RO model except use the Speigler-Kedem-Katchalsky (`"SKK"`) transport model and a `flat_sheet` module type.

Then, display the `config` dictionary and check the degrees of freedom.

# Spiral Wound: Simple vs. Complex

This cell presents a comparison of volumetric recovery and permeate concentration using these two modeling approaches.

In [ ]:
pressures = np.linspace(35e5, 85e5, 10)

##################################################################
# SIMPLE BUILD
# RO without concentration polarization, mass transfer coefficient, or pressure change

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="none",
    mass_transfer_coefficient="none",
    has_pressure_change=False,
    pressure_change_type="fixed_per_stage",
    transport_model="SD",
    module_type="spiral_wound",
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)
m.fs.RO.permeate.pressure[0].fix(atmospheric)

m.fs.RO.area.fix(membrane_area)
m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)

print(f"Degrees of freedom: {degrees_of_freedom(m)}")

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)
calculate_scaling_factors(m)

m.fs.RO.initialize()

assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

recovery1 = []
perm_conc1 = []

for p in pressures:
    m.fs.RO.feed_side.properties_in[0].pressure.fix(p)
    results = solver.solve(m)
    assert_optimal_termination(results)
    recovery1.append(value(m.fs.RO.recovery_vol_phase[0, "Liq"]))
    perm_conc1.append(
        value(m.fs.RO.permeate_side[0, 0].conc_mass_phase_comp["Liq", "TDS"])
    )


##################################################################
# COMPLEX BUILD - SD
# RO with concentration polarization, mass transfer coefficient, and pressure change

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="calculated",
    mass_transfer_coefficient="calculated",
    has_pressure_change=True,
    pressure_change_type="calculated",
    transport_model="SD",
    module_type="spiral_wound",
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)
m.fs.RO.permeate_side[0, 0].pressure.fix(atmospheric)

m.fs.RO.area.fix(membrane_area)

m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)
m.fs.RO.deltaP.fix(deltaP)
m.fs.RO.feed_side.channel_height.fix(channel_height)
m.fs.RO.feed_side.spacer_porosity.fix(spacer_porosity)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)
calculate_scaling_factors(m)

m.fs.RO.initialize()

results = solver.solve(m)
assert_optimal_termination(results)

recovery2 = []
perm_conc2 = []

for p in pressures:
    m.fs.RO.feed_side.properties_in[0].pressure.fix(p)
    results = solver.solve(m)
    assert_optimal_termination(results)
    recovery2.append(value(m.fs.RO.recovery_vol_phase[0, "Liq"]))
    perm_conc2.append(
        value(m.fs.RO.permeate_side[0, 0].conc_mass_phase_comp["Liq", "TDS"])
    )

##################################################################
# Plot results

fig, ax = plt.subplots(2, 1, figsize=(6, 7), sharex=True)
ax[0].plot(
    pressures / 1e5,
    recovery1,
    marker="o",
    color="blue",
    label="Spiral Wound: Without CP, MTC, or ∆P",
)
ax[0].plot(
    pressures / 1e5,
    recovery2,
    marker="o",
    color="red",
    label="Spiral Wound: With CP, MTC, and ∆P",
)
ax[0].set_ylabel("Volumetric Recovery")
ax[0].set_title("RO Recovery vs Feed Pressure")
ax[0].grid(visible=True)
ax[0].legend()

ax[1].plot(
    pressures / 1e5,
    perm_conc1,
    marker="o",
    color="blue",
)
ax[1].plot(
    pressures / 1e5,
    perm_conc2,
    marker="o",
    color="red",
)
ax[1].set_xlabel("Feed Pressure (bar)")
ax[1].set_ylabel("TDS Concentration in Permeate (g/L)")
ax[1].set_title("Permeate Concentration vs Feed Pressure")
ax[1].grid(visible=True)

fig.tight_layout()